# Overview of Revit Integration with Speckle

This Jupyter Notebook outlines the integration between Revit and Speckle, providing code snippets and workflows for various functionalities.

## Table of Contents

1. [Setup and Initialization](#Setup-and-Initialization)
2. [Using the Speckle Client](#Using-the-Speckle-Client)
3. [Sendind data to Speckle](#Sending-data-to-Speckle)
4. [Error Handling](#Error-Handling)
5. [Future Work](#Future-Work)


## Setup and Initialization

This section includes code to initialize the Speckle client and connect to Revit.

My preferred way to handle tokens and server details for sharing is with environment variables. The dotenv library is a great way to do this.

In [2]:
%%capture
%pip install specklepy

# dotenv is a library that allows you to load environment variables from a .env file
%pip install python-dotenv
%reload_ext dotenv
%dotenv

In [3]:
import os
HOST_SERVER = os.getenv('HOST_SERVER')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')

In [4]:
from specklepy.api.client import SpeckleClient

client = SpeckleClient(host=HOST_SERVER)  # or whatever your host is
client.authenticate_with_token(ACCESS_TOKEN)  # or whatever your token is

## Using the Speckle Client

An example of getting the stream from a speckle server:

In [5]:
stream_id = "538fcacdbe"  # or whatever your stream id is

stream = client.stream.get(stream_id)

Streams contain branches and branches contain commits. The stream object is the top level object that addresses all the data. Retreiving each is possible directly from the client with a known id or as a list.

In [6]:
branches = client.branch.list(stream_id)

branch_commits = branches[0].commits.items # commits on the first branch, which is the main branch

stream_commits = client.commit.list(stream_id) # all commits on all branches

The commits are always returned in a reverse chronological order. As such the first commit is the latest commit.

In [7]:
latest_commit = stream_commits[0] # the latest commit on the stream

latest_commit_on_branch = branch_commits[0] # the latest commit on the branch

It won't always be necessary to perform these successive commands, we can get individual objects with their known ids. If we have a specific commit we wish to use, we can get the commit object directly:

In [8]:
stream_id = "538fcacdbe"  # or whatever your stream id is
commit_id= "b8f8784a98"  # or whatever your commit id is
branch_name = "main"  # or whatever your branch name is

branch = client.branch.get(stream_id=stream_id, name=branch_name)
commit = client.commit.get(stream_id=stream_id, commit_id=commit_id )

Getting individual objects from the stream is also possible, in this case we get the commit payload object by its `referencedObject` id:

In [9]:
commit_object = client.object.get(stream_id=stream_id, object_id=commit.referencedObject)

Knowing the id of individual objects ahead of time is not always likely and retrieving them one at a time is tedious, so we probably want to get the commit object first and then get the objects from there.

As we saw a commit object retrieved by `client.commit` is itself only a pointer to a specific commit Speckle object. We can instead use the id of the commits referenced object to retrieve the committed objects themselves using Transports.

Transports are mechanisms to retrieve from a specific type of data store. Commonly used transports are to database and memory, but most commonly we will use the `ServerTransport` to retrieve from a Speckle Server.

The operations api is a library of methods that uses a given transport to perform operations on a given object. In this case we want to get the objects from a commit, so we use the `recieve` method.

In [10]:
commit_payload_id = commit.referencedObject

from specklepy.transports.server import ServerTransport
from specklepy.api import operations

transport = ServerTransport(client=client, stream_id=stream_id)
commit_payload = operations.receive(obj_id=commit_payload_id, remote_transport=transport)

In the case where you want to transform the objects received from a hierarchy, you can flatten them to a list of objects. There will be many ways to do this, my recently preferred way is to use a recursive yield function.

In this example I am getting all the objects that match certain criteria. What these rules are is up to you, but in this case I am getting all the displayable objects and Collections (for recursing through).

The yield function is a generator function, which means it will return a generator object. This object can be iterated through, and will return the next value in the sequence each time. This is useful for large lists, as it will only return the next value when it is needed, rather than all at once.

In [11]:
from typing import Any, Optional, Set, Callable, Iterator
from specklepy.objects import Base
from collections.abc import Mapping, Iterable


def should_include(obj: Any) -> bool:
    """Define a logic for what objects to include transformed data set"""
    return any(
        [
            hasattr(obj, "displayValue"),
            hasattr(obj, "speckle_type")
            and obj.speckle_type == "Objects.Organization.Collection",
            hasattr(obj, "displayStyle"),
        ]
    )


def flatten(
    obj: Any,
    visited: Optional[Set[Any]] = None,
    include_func: Callable[[Any], bool] = should_include,
) -> Iterator[Any]:
    """Flattens nested object structures based on given criteria.

    Parameters:
        obj: The object to flatten.
        visited: Set of objects that have already been visited to avoid circular references.
        include_func: Function to determine if an object should be included in the output.
    """
    if visited is None:
        visited = set()

    if obj in visited:
        return

    visited.add(obj)

    if include_func(obj):
        yield obj

    props = obj.__dict__ if hasattr(obj, "__dict__") else {}

    for prop in props:
        value = getattr(obj, prop)

        if value is None:
            continue

        if isinstance(value, Base):
            yield from flatten(value, visited, include_func)

        elif isinstance(value, Mapping):
            for dict_value in value.values():
                if isinstance(dict_value, Base):
                    yield from flatten(dict_value, visited, include_func)

        elif isinstance(value, Iterable):
            for list_value in value:
                if isinstance(list_value, Base):
                    yield from flatten(list_value, visited, include_func)

Retrieving the objects from the generator is as simple as casting to a list:

In [12]:
all_objects = list(flatten(commit_payload))

### NB: This code is simplistic and may not cover the way that individual objects are hosted by other, or have more complex releationships between them.

At this point, use that flat list of objects to do whatever you want with them. 

## Sending data to Speckle

Assuming some other work has been done to the `all_objects` list, we can send the data back to Speckle, also using the client.

By convention most of the Speckle desktop connectors top level commit object is a Collection class. This is a simple class that contains a list of objects in an `elements` property, it can convey intent and identity with `name` and `collectionType`. This is not a strict requirement, but many of the Speckle tools have helper functions that can utilse the Colllection->elements->objects structure.

In [13]:
from specklepy.objects.other import Collection

new_commit = Collection(collectionType="flat_list", name="Processed Data", elements=[])

new_commit.elements = all_objects

The top level content of the new commit is now our flat list. We can send this to the server using the `send` method of the operations api.

In [14]:
# Send the points using a specific transport
newHash = operations.send(base=new_commit, transports=[transport])

The `transports` prop is a list of `specklepy.transports.base.BaseTransport` objects. You can create your own transport by inheriting from `specklepy.transports.base.BaseTransport` giving the potential to apply the same transformation to multiple destinations. By default it will send to whatever transports you set AND the local cache. This can be overridden, but means if you dont specify a transport it will always send to the local cache.

It's worth describe a little of what that single `send` method does behind the scenes.

It will pass whatever base object you give it through the serializer. This will traverse the structure you pass it and any detachable objects, chunkable properties or other cases it handles will be compared individually with those stored on the server, and only those that have changed will be sent. They will each be hashed to get a unique id. This is done recursively, so if you have a list of objects, each object will be compared individually.

The hashed ids will be exchanged in place of the objects as a referenceObject in you data sent and when all are traversed, the parents are sent and so on. The top of the tree Base object finally returns its id.

While commits can be addressed by their id alone on receive, to send data we should specify the branch we wish to send to. That branch will need to exist in advance. You can get a list of exsiting branches and select one, or create a new one.

In [15]:
from specklepy.api.resources import Branch

def new_branch(stream_id: str, branch_name: str) -> Branch:
    """Creates a new branch in a stream.

    Parameters:
        stream_id: The stream id.
        branch_name: The name of the new branch.
    """
    try:
        client.branch.create(stream_id, name=branch_name)
    finally:
        return client.branch.get(stream_id, name=branch_name)

ImportError: cannot import name 'Branch' from 'specklepy.api.resources' (/home/jsdbroughton/repos/speckle/speckle-examples/python/speckle-jupyter-notebooks/.venv/lib/python3.8/site-packages/specklepy/api/resources/__init__.py)

Calling that function will always return a Branch object whether it already exists or not. If it already exists, it will return the existing branch, if not it will create a new one and return that.

In [ ]:
branch = new_branch(stream_id=stream_id, branch_name="jssadb")

Branch(id='d540ed2cbc', name='jssadb', description='No description provided', commits=Commits(totalCount=3, cursor=datetime.datetime(2023, 9, 15, 9, 43, 11, 762000, tzinfo=datetime.timezone.utc), items=[Commit( id: a88f62984c, message: Commit, referencedObject: 7695d1c3eae5a89ee5ed5cd0974529d9, authorName: Jonathon Broughton, branchName: jssadb, createdAt: 2023-09-15 09:58:01.271000+00:00 ), Commit( id: 14febb9340, message: Commit, referencedObject: 7695d1c3eae5a89ee5ed5cd0974529d9, authorName: Jonathon Broughton, branchName: jssadb, createdAt: 2023-09-15 09:47:35.265000+00:00 ), Commit( id: 1ac90f9426, message: Commit, referencedObject: 7695d1c3eae5a89ee5ed5cd0974529d9, authorName: Jonathon Broughton, branchName: jssadb, createdAt: 2023-09-15 09:43:11.762000+00:00 )]))

Now we can send the commit to the server. The objects data already exists on the server at this point, the commit we make now registers the data collection as a commit entry.

This will needs only the id for the parent Base object and a target stream and branch. The message is optional, but can be useful to describe the commit. It will return a commit id.

In [ ]:

commit_id = client.commit.create(
        stream_id,
        newHash,
        branch.name,
        message="Commit",
    )

## Revit Specfic notes

You'll notice that nothing so far is different from any other data connector. Nothing Revit specific at all.

Indeed, the Revit connector is just a wrapper around the C# client that specklepy mirrors. All the Revit specific code will be in that Connector.

Possible areas to be aware of as a data wrangler in Revit will be:
1. The general pattern of the data commited from Revit Connector
2. The specifics of Revit Paramerter properties per object
3. Instances vs Types
4. Hosted elements



In the walkthrough above, the data was sent to the server as a `Collection` object and the data of interest within the `elements` field. This is a common pattern for the desktop connectors, but not a strict requirement. The Revit connector will send the data as a `Collection` object where the `collectionType` is "`Speckle.Core.Models.Collection:Objects.Organization.Model`", but the data of interest will be in the `speckle_type` field. This is to allow for the `speckle_type` to be used as a Revit category, which is a useful way to filter the data in Revit.

So far we have been looking at the totality of the commit data being sent, which could be an entire federated model with many thousands of objects. This is not always desirable, and we may want to send only a subset of the data. This is possible by filtering the data before sending it.

Fortunately, the GraphQL API that enables filtering is also available directly with the existing SpecklePy client.


In [16]:
stream_id = "538fcacdbe"  # or whatever your stream id is
commit_id= "b8f8784a98"  # or whatever your commit id is
branch_name = "main"  # or whatever your branch name is

branch = client.branch.get(stream_id=stream_id, name=branch_name)
commit = client.commit.get(stream_id=stream_id, commit_id=commit_id )

In [49]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

commit_referenced_object = commit.referencedObject

gql_client = Client(
    transport=RequestsHTTPTransport(
        url=f"{HOST_SERVER}/graphql"
    )
)

query = gql(
    """ query Commit($stream_id: String!, $commit_referenced_object: String!) { 
            stream(id: $stream_id) { 
            object(id: $commit_referenced_object) {
              children(limit: 1000 select: ["level.name", "level.parameters.LEVEL_ELEV.value"]) {
               objects {
                  data
                }
              }
           }
         }
       } """
)
params = {
  "stream_id": stream_id, 
  "commit_referenced_object": commit_referenced_object
}

received_data = gql_client.execute(query, variable_values=params)

received_data

{'stream': {'object': {'children': {'objects': [{'data': {'level': {'name': 'Foundation',
        'parameters': {'LEVEL_ELEV': {'value': -1200}}}}},
     {'data': {'level': {'name': 'Level 1',
        'parameters': {'LEVEL_ELEV': {'value': 0}}}}},
     {'data': {'level': {'name': 'Level Lower',
        'parameters': {'LEVEL_ELEV': {'value': -550}}}}},
     {'data': {'level': {'name': 'Level 2',
        'parameters': {'LEVEL_ELEV': {'value': 3000}}}}},
     {'data': {'level': {'name': 'Level 2',
        'parameters': {'LEVEL_ELEV': {'value': 3000}}}}},
     {'data': {'level': {'name': 'Level Lower',
        'parameters': {'LEVEL_ELEV': {'value': -550}}}}},
     {'data': {'level': {'name': 'Level 2',
        'parameters': {'LEVEL_ELEV': {'value': 3000}}}}},
     {'data': {'level': {'name': 'Level 1',
        'parameters': {'LEVEL_ELEV': {'value': 0}}}}},
     {'data': {'level': {'name': 'Level Lower',
        'parameters': {'LEVEL_ELEV': {'value': -550}}}}},
     {'data': {'level': {'nam

In [ ]:

# Initialize an empty set to store unique levels
unique_levels = set()

# Navigate through the nested structure to extract level names and elevations
if received_data and 'stream' in received_data and 'object' in received_data['stream']:
    objects = received_data['stream']['object'].get('children', {}).get('objects', [])
    for obj in objects:
      level_data = obj.get('data', {}).get('level', {})
      name = level_data.get('name', 'Unknown')
      elevation = level_data.get('parameters', {}).get('LEVEL_ELEV', {}).get('value', 'Unknown')
      unique_levels.add((name, elevation))

# Convert the set to a list for better readability
unique_levels_list = list(unique_levels)

# Sort the list of unique levels by their elevation
sorted_unique_levels = sorted(unique_levels_list, key=lambda x: x[1])

# Show the sorted list
sorted_unique_levels